## Google Drive Mount

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My\ Drive/bamboo_generator

/content/gdrive/My Drive/bamboo_generator


In [3]:
ls

dataloader.py      figures/       rollout.py     seq_gan_MLE.ipynb
dataloader.pyc     generator.py   rollout.pyc    sequence_gan.py
discriminator.py   generator.pyc  save/
discriminator.pyc  README.md      seq_gan.ipynb


In [0]:
import numpy as np
import tensorflow as tf
import random
from dataloader import Gen_Data_loader, Dis_dataloader
from generator import Generator
from discriminator import Discriminator
from rollout import ROLLOUT
import cPickle
import timeit

In [0]:

#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
EMB_DIM = 100 # embedding dimension
HIDDEN_DIM = 32 # hidden state dimension of lstm cell
SEQ_LENGTH = 30 # sequence length
START_TOKEN = 0
SEED = 88
BATCH_SIZE = 64

#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 100
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64


#########################################################################################
#  Basic Training Parameters
#########################################################################################
positive_file = 'save/input/positive_sample.txt'
negative_file = 'save/input/generator_sample.txt'
eval_file = 'save/input/eval_file.txt'
generated_num = 10000
ckpt_path = 'save/train1'
vocab_size = 30210

#########################################################################################
# About an epoch
#########################################################################################
PRE_EPOCH_NUM = 150 # supervise (maximum likelihood estimation) epochs = generated
PRE_DIS_EPCOCH = 1 # pre-training for discriminator
TOTAL_BATCH = 1 # epoch for Adversaraila-Training

In [0]:
def generate_samples(sess, trainable_model, batch_size, generated_num, output_file):
    # Generate Samples
    generated_samples = []
    for _ in range(int(generated_num / batch_size)):
        generated_samples.extend(trainable_model.generate(sess))

    with open(output_file, 'w') as fout:
        for poem in generated_samples:
            buffer = ' '.join([str(x) for x in poem]) + '\n'
            fout.write(buffer)


def pre_train_epoch(sess, trainable_model, data_loader):
    # Pre-train the generator using MLE for one epoch
    supervised_g_losses = []
    data_loader.reset_pointer()

    for it in xrange(data_loader.num_batch):
        batch = data_loader.next_batch()
        _, g_loss = trainable_model.pretrain_step(sess, batch)
        supervised_g_losses.append(g_loss)

    return np.mean(supervised_g_losses)


In [0]:
# with train_loss version
start = timeit.default_timer()

random.seed(SEED)
np.random.seed(SEED)
assert START_TOKEN == 0

gen_data_loader = Gen_Data_loader(BATCH_SIZE)
likelihood_data_loader = Gen_Data_loader(BATCH_SIZE) # For testing
vocab_size = 30210
dis_data_loader = Dis_dataloader(BATCH_SIZE)

generator = Generator(vocab_size, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, SEQ_LENGTH, START_TOKEN)

discriminator = Discriminator(sequence_length=SEQ_LENGTH, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim, 
                            filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, l2_reg_lambda=dis_l2_reg_lambda)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

# Get batches from positive samples 
gen_data_loader.create_batches(positive_file)

# Save variables
saver = tf.train.Saver()

log = open('save/experiment-log.txt', 'w')
#  pre-train generator
print 'Start pre-training...'
log.write('pre-training...\n')
for epoch in xrange(PRE_EPOCH_NUM):
    loss = pre_train_epoch(sess, generator, gen_data_loader)
    if epoch % 5 == 0:
        generate_samples(sess, generator, BATCH_SIZE, generated_num, eval_file)
        likelihood_data_loader.create_batches(eval_file)

        print 'pre-train epoch ', epoch, 'train_loss ', loss
        buffer = 'epoch:\t'+ str(epoch) + '\tnll:\t' + str(loss) + ' \n'
        log.write(buffer)

    saver.save(sess, ckpt_path)
    
stop = timeit.default_timer()

print 'Time : ', stop - start

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Start pre-training...
pre-train epoch  0 train_loss  5.4674697
pre-train epoch  5 train_loss  3.9428532
pre-train epoch  10 train_loss  3.5363038
pre-train epoch  15 train_loss  3.3652523
pre-train epoch  20 train_loss  3.2484496
pre-train epoch  25 train_loss  3.1647017
pre-train epoch  30 train_loss  3.1006021
pre-train epoch  35 train_loss  3.0459907
pre-train epoch  40 train_loss  2.9979994
pre-train epoch  45 train_loss  2.9613495
pre-train epoch  50 train_loss  2.9283037
pre-train epoch  55 train_loss  2.903041
pre-train epoch  60 train_loss  2.8799372
pre-train epoch  65 train_loss  2.8655899
pre-train epoch  70 train_loss  2.8596137
